<a href="https://colab.research.google.com/github/Kamiliaadil/LLMs-Final-Project-Fine-Tuning-Lora-for-Mental-Health-Counseling-with-Llama2/blob/main/Mental_Health_Counseling_with_Llama2_Fine_Tuning_Lora.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**In this notebook, I'm  working with a dataset from online counseling platforms containing user questions and psychologist responses. Using the llama architecture in Python, I'll develop a fine-tuned language model. The objective is to train the model on counseling data, enabling it to generate helpful responses to user queries.**

In [ ]:

!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 17.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.9 MB/s eta 0:00:00


In [ ]:
!pip install accelerate==0.21.0
!pip install peft==0.4.0
!pip install bitsandbytes==0.40.2
!pip install transformers==4.31.0
!pip install trl==0.4.7

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 882.6 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.5/92.5 MB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 41.0 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.15.0
    Uninstalling tokenizers-0.15.0:
      Successfully uninstalled tokenizers-0.15.0
  Attempting uninstall: transformers
    Found existing installation: transformers 4.35.2
    Uninstalling transformers-4.35.2:
      Successfully uninstalled transformers-4.35.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.4/77.4 kB 1.7 MB/s eta 0:00:00


In [ ]:
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer

In [ ]:

# The model that you want to train from the Hugging Face hub
model_name = "NousResearch/Llama-2-7b-chat-hf"

# The instruction dataset to use
dataset_name = "Amod/mental_health_counseling_conversations"

# Fine-tuned model name
new_model = "llama-2-7b-mental_health"

In [ ]:
# Load your dataset
#dataset = load_dataset("Amod/mental_health_counseling_conversations")



In [ ]:
#print(dataset.column_names)

NameError: ignored

In [ ]:
# Extract 'text' from 'Context' and 'Response'
#new_dataset = dataset.map(lambda example: {'text': example['Context'] + ' ' + example['Response']}, remove_columns=['Context', 'Response'])

In [ ]:

# The model that you want to train from the Hugging Face hub
#model_name = "NousResearch/Llama-2-7b-chat-hf"

# Fine-tuned model name
#new_model = "llama-2-7b-miniguanaco"

In [ ]:

################################################################################
# QLoRA parameters
################################################################################

# LoRA attention dimension
lora_r = 64

# Alpha parameter for LoRA scaling
lora_alpha = 16

# Dropout probability for LoRA layers
lora_dropout = 0.1


In [ ]:

################################################################################
# bitsandbytes parameters
################################################################################

# Activate 4-bit precision base model loading
use_4bit = True

# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "float16"

# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"

# Activate nested quantization for 4-bit base models (double quantization)
use_nested_quant = False

In [ ]:
################################################################################
# TrainingArguments parameters
################################################################################

# Output directory where the model predictions and checkpoints will be stored
output_dir = "./results"

# Number of training epochs
num_train_epochs = 1

# Enable fp16/bf16 training (set bf16 to True with an A100)
fp16 = False
bf16 = False

# Batch size per GPU for training
per_device_train_batch_size = 4

# Batch size per GPU for evaluation
per_device_eval_batch_size = 4

# Number of update steps to accumulate the gradients for
gradient_accumulation_steps = 1

# Enable gradient checkpointing
gradient_checkpointing = True

# Maximum gradient normal (gradient clipping)
max_grad_norm = 0.3

# Initial learning rate (AdamW optimizer)
learning_rate = 2e-4

# Weight decay to apply to all layers except bias/LayerNorm weights
weight_decay = 0.001

# Optimizer to use
optim = "paged_adamw_32bit"

# Learning rate schedule (constant a bit better than cosine)
lr_scheduler_type = "constant"

# Number of training steps (overrides num_train_epochs)
max_steps = -1

# Ratio of steps for a linear warmup (from 0 to learning rate)
warmup_ratio = 0.03

# Group sequences into batches with same length
# Saves memory and speeds up training considerably
group_by_length = True

# Save checkpoint every X updates steps
save_steps = 25

# Log every X updates steps
logging_steps = 25


In [ ]:
# SFT parameters

# Maximum sequence length to use
max_seq_length = None

# Pack multiple short examples in the same input sequence to increase efficiency
packing = False

# Load the entire model on the GPU 0
device_map = {"": 0}


In [ ]:

# Load dataset (you can process it here)
dataset = load_dataset(dataset_name, split="train")

# Load tokenizer and model with QLoRA configuration
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
# Load dataset (you can process it here)
dataset = load_dataset(dataset_name, split="train")

# Extract 'text' from 'Context' and 'Response'
dataset = dataset.map(
    lambda example: {'text': example['Context'] + ' ' + example['Response']},
    remove_columns=['Context', 'Response']
)

# Load tokenizer and model with QLoRA configuration
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

Map:   0%|          | 0/3512 [00:00<?, ? examples/s]

In [ ]:
# Load base model
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map=device_map
)
model.config.use_cache = False
model.config.pretraining_tp = 1

config.json:   0%|          | 0.00/583 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/179 [00:00<?, ?B/s]

In [ ]:

# Load LLaMA tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

tokenizer_config.json:   0%|          | 0.00/746 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/435 [00:00<?, ?B/s]

In [ ]:
# Load LoRA configuration
peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
)

In [ ]:
# Set training parameters
training_arguments = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    fp16=fp16,
    bf16=bf16,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=group_by_length,
    lr_scheduler_type=lr_scheduler_type,
    report_to="tensorboard"
)

In [ ]:

# Set supervised fine-tuning parameters
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
    packing=packing,
)

/usr/local/lib/python3.10/dist-packages/peft/utils/other.py:102: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:159: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


Map:   0%|          | 0/3512 [00:00<?, ? examples/s]

In [ ]:
# Train model
trainer.train()

# Save trained model
trainer.model.save_pretrained(new_model)

You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
25,2.328900
50,2.311300
75,2.159100
100,2.198400
125,2.198300
150,2.229100
175,2.167400
200,2.180600
225,2.073700
250,2.140100


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.

In [ ]:
# Ignore warnings
logging.set_verbosity(logging.CRITICAL)
prompt = "I am feeling stressed and anxious"
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200)
result = pipe(f"[INST] {prompt} [/INST]")
print(result[0]['generated_text'])

[INST] I am feeling stressed and anxious [/INST]  Sorry to hear that you're feeling stressed and anxious. hopefully, this will help: Stress and anxiety are very common and can be caused by a wide range of things. The first step is to identify what is causing your stress and anxiety. Once you have identified the cause, you can start to think about ways to manage it. Here are some ideas: 1. Exercise: Exercise is a great way to reduce stress and anxiety. 2. Mindfulness: Mindfulness is a great way to reduce stress and anxiety. 3. Breathing exercises: Breathing exercises can help reduce stress and anxiety. 4. Relaxation techniques: Relaxation techniques can help reduce stress and anxiety. 5. Meditation: Meditation can help reduce stress


R1: Sorry to hear that you're feeling stressed and anxious. hopefully, this will help: Stress and anxiety are very common and can be caused by a wide range of things. The first step is to identify what is causing your stress and anxiety. Once you have identified the cause, you can start to think about ways to manage it. Here are some ideas: 1. Exercise: Exercise is a great way to reduce stress and anxiety. 2. Mindfulness: Mindfulness is a great way to reduce stress and anxiety. 3. Breathing exercises: Breathing exercises can help reduce stress and anxiety. 4. Relaxation techniques: Relaxation techniques can help reduce stress and anxiety. 5. Meditation: Meditation can help reduce stress

In [ ]:
prompt = "I'm drowning in studies. Each assignment feels like a mountain, and exams are suffocating. The pressure to perform is overwhelming. It's a never-ending treadmill, and the stress is hurting my well-being"
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200)
result = pipe(f"[INST] {prompt} [/INST]")
print(result[0]['generated_text'])

[INST] I'm drowning in studies. Each assignment feels like a mountain, and exams are suffocating. The pressure to perform is overwhelming. It's a never-ending treadmill, and the stress is hurting my well-being [/INST]  I'm sorry to hear that you're feeling overwhelmed by your studies. It's important to remember that you are not alone in feeling this way. Many students feel this way at some point in their academic careers. It's important to remember that you are not defined by your academic performance. You are a unique individual with many strengths and talents. It's important to find balance in your life. I would suggest that you speak with your academic advisor about your concerns. They may be able to help you find ways to manage your stress. You may also want to consider speaking with a counselor or therapist. They


R2: I'm sorry to hear that you're feeling overwhelmed by your studies. It's important to remember that you are not alone in feeling this way. Many students feel this way at some point in their academic careers. It's important to remember that you are not defined by your academic performance. You are a unique individual with many strengths and talents. It's important to find balance in your life. I would suggest that you speak with your academic advisor about your concerns. They may be able to help you find ways to manage your stress. You may also want to consider speaking with a counselor or therapist. They

In [ ]:
prompt = "I feel worthless"
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200)
result = pipe(f"[INST] {prompt} [/INST]")
print(result[0]['generated_text'])

[INST] I feel worthless [/INST]  Sorry to hear that you feel worthless. hopefully this will help: The way we feel about ourselves is often based on our past experiences and the way we perceive ourselves in the present.  If you are feeling worthless, it is important to understand that this is not a reflection of your true worth.  It is a reflection of the way you are feeling about yourself right now.  It is important to understand that you are not your feelings.  You are not your thoughts.  You are not your past.  You are not your future.  You are a person who is experiencing a feeling of worthlessness.  This is a feeling that you can change.  You can change it by changing the way you think about yourself.  You can change it by changing the way you feel about yourself.  You can change it by changing the way you act.  You can change it by changing the way


R3: Sorry to hear that you feel worthless. hopefully this will help: The way we feel about ourselves is often based on our past experiences and the way we perceive ourselves in the present.  If you are feeling worthless, it is important to understand that this is not a reflection of your true worth.  It is a reflection of the way you are feeling about yourself right now.  It is important to understand that you are not your feelings.  You are not your thoughts.  You are not your past.  You are not your future.  You are a person who is experiencing a feeling of worthlessness.  This is a feeling that you can change.  You can change it by changing the way you think about yourself.  You can change it by changing the way you feel about yourself.  You can change it by changing the way you act.  You can change it by changing the way


In [ ]:
# Ignore warnings
logging.set_verbosity(logging.CRITICAL)
prompt = "I think people don't really like me or value me. It's tough."
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200)
result = pipe(f"[INST] {prompt} [/INST]")
print(result[0]['generated_text'])

[INST] I think people don't really like me or value me. It's tough. [/INST]  It sounds like you are feeling pretty down about yourself. It's important to remember that you are not alone in feeling this way. Many people struggle with feeling like they are not valued or liked by others. It's important to remember that you are not defined by how others see you. You are a unique and special person with your own strengths and weaknesses. It's okay to have flaws and it's okay to not be perfect. It's okay to be yourself. It's okay to be different. It's okay to not fit in. It's okay to be you. It's okay to be you. It's okay to be you. It's okay to be you. It's okay to be you. It's okay to be you. It's okay to


R4:  It sounds like you are feeling pretty down about yourself. It's important to remember that you are not alone in feeling this way. Many people struggle with feeling like they are not valued or liked by others. It's important to remember that you are not defined by how others see you. You are a unique and special person with your own strengths and weaknesses. It's okay to have flaws and it's okay to not be perfect. It's okay to be yourself. It's okay to be different. It's okay to not fit in. It's okay to be you. It's okay to be you. It's okay to be you. It's okay to be you. It's okay to be you. It's okay to be you. It's okay to

In [ ]:
prompt = "I'm alone."
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200)
result = pipe(f"[INST] {prompt} [/INST]")
print(result[0]['generated_text'])

[INST] I'm alone. [/INST]  Sorry to hear that you're feeling alone. It can be difficult to feel disconnected from others, especially when we're feeling down or struggling with difficult emotions. It's important to remember that you're not alone in feeling this way. Many people struggle with feeling alone, and it's okay to feel this way. It's important to reach out to others when we're feeling this way, whether it's a friend, family member, or mental health professional. They can offer support and help us feel less alone. If you're feeling overwhelmed and don't know where to turn, consider reaching out to a crisis hotline or text line. They can provide immediate support and help you feel less alone. Remember, you're not alone, and there are people who care about you and want to help. If you're feeling overwhelmed and don


R5:Sorry to hear that you're feeling alone. It can be difficult to feel disconnected from others, especially when we're feeling down or struggling with difficult emotions. It's important to remember that you're not alone in feeling this way. Many people struggle with feeling alone, and it's okay to feel this way. It's important to reach out to others when we're feeling this way, whether it's a friend, family member, or mental health professional. They can offer support and help us feel less alone. If you're feeling overwhelmed and don't know where to turn, consider reaching out to a crisis hotline or text line. They can provide immediate support and help you feel less alone. Remember, you're not alone, and there are people who care about you and want to help. If you're feeling overwhelmed and don

In [ ]:
prompt = "Feeling like I'm stuck in a deep, dark tunnel with no light at the end. Each day is a battle against a heavy fog of sadness and worry that just won't lift. It's like being caught in a storm of negative thoughts, and I can't find shelter."
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200)
result = pipe(f"[INST] {prompt} [/INST]")
print(result[0]['generated_text'])

/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:61: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


[INST] Feeling like I'm stuck in a deep, dark tunnel with no light at the end. Each day is a battle against a heavy fog of sadness and worry that just won't lift. It's like being caught in a storm of negative thoughts, and I can't find shelter. [/INST]  It sounds like you are experiencing a lot of distress and are feeling overwhelmed by the negative thoughts and emotions that you are experiencing. It's important to remember that these feelings are temporary and that they will pass. In the meantime, there are a few things that you can do to help manage your symptoms:1. Practice self-compassion: It's important to be kind to yourself and to remember that you are not alone in this struggle. Try to be gentle with yourself and to remind yourself that you are doing the best that you can.2. Practice mindfulness:


R6:  It sounds like you are experiencing a lot of distress and are feeling overwhelmed by the negative thoughts and emotions that you are experiencing. It's important to remember that these feelings are temporary and that they will pass. In the meantime, there are a few things that you can do to help manage your symptoms:1. Practice self-compassion: It's important to be kind to yourself and to remember that you are not alone in this struggle. Try to be gentle with yourself and to remind yourself that you are doing the best that you can.2. Practice mindfulness:

In [ ]:
prompt = "I often end up putting others first, really going out of my way to be there for them. But, you know, it can get tough when it seems like nobody notices or appreciates what I do. It's like a constant effort that sometimes goes unnoticed."
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200)
result = pipe(f"[INST] {prompt} [/INST]")
print(result[0]['generated_text'])

[INST] I often end up putting others first, really going out of my way to be there for them. But, you know, it can get tough when it seems like nobody notices or appreciates what I do. It's like a constant effort that sometimes goes unnoticed. [/INST]  It sounds like you are a very caring person who is always looking out for others. It's great that you are able to be there for them, but it's also important to remember that you can't always be there for everyone. It's okay to take a step back and focus on yourself sometimes. It's important to remember that you are not a bottomless pit of giving. You can't give everything to everyone and still have something left for yourself. It's okay to say no sometimes and to put yourself first. It's also important to remember that you can't control how others feel or act. You can


R7:  It sounds like you are a very caring person who is always looking out for others. It's great that you are able to be there for them, but it's also important to remember that you can't always be there for everyone. It's okay to take a step back and focus on yourself sometimes. It's important to remember that you are not a bottomless pit of giving. You can't give everything to everyone and still have something left for yourself. It's okay to say no sometimes and to put yourself first. It's also important to remember that you can't control how others feel or act. You can

In [ ]:
prompt = "I feel lost, unable to think or find purpose."
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200)
result = pipe(f"[INST] {prompt} [/INST]")
print(result[0]['generated_text'])

[INST] I feel lost, unable to think or find purpose. [/INST]  Feeling lost and unable to think or find purpose can be a very difficult and frustrating experience. It is important to remember that these feelings are normal and can be a part of the healing process. It is important to remember that you are not alone in this experience. Many people have felt this way at some point in their lives. It is important to remember that you are not alone in this experience. Many people have felt this way at some point in their lives. It is important to remember that you are not alone in this experience. Many people have felt this way at some point in their lives. It is important to remember that you are not alone in this experience. Many people have felt this way at some point in their lives. It is important to remember that you are not alone in this experience. Many people have felt this way at some point in their lives


R8:   Feeling lost and unable to think or find purpose can be a very difficult and frustrating experience. It is important to remember that these feelings are normal and can be a part of the healing process. It is important to remember that you are not alone in this experience. Many people have felt this way at some point in their lives. It is important to remember that you are not alone in this experience. Many people have felt this way at some point in their lives. It is important to remember that you are not alone in this experience. Many people have felt this way at some point in their lives. It is important to remember that you are not alone in this experience. Many people have felt this way at some point in their lives. It is important to remember that you are not alone in this experience. Many people have felt this way at some point in their lives

In [ ]:
prompt = "I feel depressed."
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200)
result = pipe(f"[INST] {prompt} [/INST]")
print(result[0]['generated_text'])

[INST] I feel depressed. [/INST]  Sorry to hear that you're feeling depressed.  I'm not a mental health professional, but I can offer some suggestions that may help.  First, I would encourage you to talk to someone you trust about how you're feeling.  This could be a friend, family member, or a mental health professional.  Talking about your feelings can help you feel better and can also help you identify the root cause of your depression.  If you're not comfortable talking to someone you know, you can also call a crisis hotline.  There are many crisis hotlines available, and they are free and confidential.  You can also text the Crisis Text Line at 741741.  If you're feeling suicidal, please call 911 or go to the nearest emergency room.  If you're feeling overwhelmed and


R9: Sorry to hear that you're feeling depressed.  I'm not a mental health professional, but I can offer some suggestions that may help.  First, I would encourage you to talk to someone you trust about how you're feeling.  This could be a friend, family member, or a mental health professional.  Talking about your feelings can help you feel better and can also help you identify the root cause of your depression.  If you're not comfortable talking to someone you know, you can also call a crisis hotline.  There are many crisis hotlines available, and they are free and confidential.  You can also text the Crisis Text Line at 741741.  If you're feeling suicidal, please call 911 or go to the nearest emergency room.  If you're feeling overwhelmed and